In [4]:
import numpy as np
import torch
import torch.nn.functional
from e3nn import o3
from matplotlib import pyplot as plt
import ase.io
from ase.visualize import view
from scipy.spatial.transform import Rotation
import warnings
warnings.filterwarnings("ignore")
from ase.io import read, write
from matplotlib import pyplot as plt
import numpy as np
from aseMolec import extAtoms as ea
from ase.data import atomic_numbers
from mace import data, modules, tools
from mace.tools import torch_geometric
torch.set_default_dtype(torch.float64)


db = read('../data/DMABN_combined.xyz', ':')
E0s = ea.get_E0(read('../data/E0s_S0.xyz', ':'), tag='0')
db = read('../data/DMABN_combined_converted_S0.xyz', ':')
z_table = tools.AtomicNumberTable([atomic_numbers[symbol] for symbol in E0s.keys()])

default_model_config = dict(
        num_elements=3,  # number of chemical elements
        atomic_energies=list(E0s.values()),  # atomic energies used for normalisation
        avg_num_neighbors=8,  # avg number of neighbours of the atoms, used for internal normalisation of messages
        atomic_numbers=z_table.zs,  # atomic numbers, used to specify chemical element embeddings of the model
        r_max=6,  # cutoff
        num_bessel=8,  # number of radial features
        num_polynomial_cutoff=6,  # smoothness of the radial cutoff
        max_ell=2,  # expansion order of spherical harmonic adge attributes
        num_interactions=2,  # number of layers, typically 2
        interaction_cls_first=modules.interaction_classes[
            "RealAgnosticResidualInteractionBlock"
        ],  # interation block of first layer
        interaction_cls=modules.interaction_classes[
            "RealAgnosticResidualInteractionBlock"
        ],  # interaction block of subsequent layers
        hidden_irreps=o3.Irreps("8x0e + 8x1o"),  # 8: number of embedding channels, 0e, 1o is specifying which equivariant messages to use. Here up to L_max=1
        correlation=3,  # correlation order of the messages (body order - 1)
        MLP_irreps=o3.Irreps("16x0e"),  # number of hidden dimensions of last layer readout MLP
        gate=torch.nn.functional.silu,  # nonlinearity used in last layer readout MLP
    )

model = modules.XstatesMACE(**default_model_config)

configs = [data.config_from_atoms(atoms) for atoms in db]

# we handle configurations using the AtomicData class
data_loader = torch_geometric.dataloader.DataLoader(
    dataset=[
        data.AtomicData.from_config(config, z_table=z_table, cutoff=6.0)
        for config in configs
    ],
    batch_size=3,
    shuffle=False,
    drop_last=False
)

model(list(data_loader)[0])

AttributeError: module 'mace.modules' has no attribute 'XstatesMACE'